Clone the tutorial repo

In [1]:
import world
import utils
from world import cprint
import register
import torch
import numpy as np
from tensorboardX import SummaryWriter
import time
import Procedure
from os.path import join
# ==============================
utils.set_seed(world.seed)
print(">>SEED:", world.seed)
# ==============================

from register import dataset
from torch import nn

from dataloader import BasicDataset
import dataloader
# import odeblock as ode


██╗  ████████╗    ██████╗  ██████╗███████╗
██║  ╚══██╔══╝   ██╔═══██╗██╔════╝██╔════╝
██║     ██║█████╗██║   ██║██║     █████╗  
██║     ██║╚════╝██║   ██║██║     ██╔══╝  
███████╗██║      ╚██████╔╝╚██████╗██║     
╚══════╝╚═╝       ╚═════╝  ╚═════╝╚═╝     
                                                      

Current cuda device  0
loading [../data/gowalla]
810128 interactions for training
217242 interactions for testing
gowalla Sparsity : 0.0008396216228570436
gowalla is ready to go
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'K': 4,
 'bigdata': False,
 'bpr_batch_size': 2048,
 'decay': 0.0001,
 'dropout': 0,
 'dual_res': False,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'learnable_time': False,
 'lightGCN_n_layers': 3,
 'lr': 0.001,
 'lr_time': 0.0001,
 'multicore': 0,
 'pretrain': 0,
 'pretrained_file_name': 'ltocf',
 'solver': 'euler',
 'test_u_batch_size': 100,
 'time_split': 4}
cores for test: 32
comment: lt-ncf
tensorboard: 1
LOAD: 0
Weight path: 

In [2]:
if world.adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

class ODEFunction(nn.Module):
    """
       ## linear GCN (non-time-dependent) in ODE function
    """
    def __init__(self, Graph):
        super(ODEFunction, self).__init__()
        self.g = Graph

    def forward(self, t, x):
        """
        ## linear GCN (non-time-dependent) in ODE function
        
        \begin{align}
        $ \boldsymbol{E}_{k} = \boldsymbol{A}\boldsymbol{E}_{k-1} $
        \end{align}
        """
        out = torch.sparse.mm(self.g, x)
        return out

class ODEBlock(nn.Module):
    def __init__(self, odeFunction, solver, init_time, final_time):
        super(ODEBlock, self).__init__()
        self.odefunc = odeFunction
        self.integration_time = torch.tensor([init_time,final_time]).float()
        self.solver = solver

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(func=self.odefunc, y0=x, t=self.integration_time, method=self.solver)
        return out[1]

def ODETimeSplitter(num_split, K):
        eta = K / num_split
        return [i*eta for i in range(1, num_split)]
        
def ODETimeSetter(num_split, K):
        eta = K/ num_split
        return [torch.tensor([i*eta], dtype=torch.float32, requires_grad=True, device='cuda') for i in range(1, num_split)]


In [3]:
#FIXED
class LTOCF(nn.Module):
    def __init__(self, 
                 config:dict, 
                 dataset:BasicDataset):
        super(LTOCF, self).__init__()
        self.config = config
        self.dataset : dataloader.BasicDataset = dataset
        self.__init_weight()
        self.__init_ode()


    def __init_weight(self):
        self.num_users  = self.dataset.n_users
        self.num_items  = self.dataset.m_items
        self.latent_dim = self.config['latent_dim_rec']
        self.n_layers = self.config['lightGCN_n_layers']
        self.keep_prob = self.config['keep_prob']
        self.A_split = self.config['A_split']
        self.embedding_user = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.latent_dim)
        if self.config['pretrain'] == 0:
            nn.init.normal_(self.embedding_user.weight, std=0.1)
            nn.init.normal_(self.embedding_item.weight, std=0.1)
            world.cprint('use NORMAL distribution initilizer')
        else:
            print('use pretarined data')
        self.f = nn.Sigmoid()
        self.Graph = self.dataset.getSparseGraph()
        print(f"lgn is already to go(dropout:{self.config['dropout']})")
    
    def __init_ode(self):
        self.time_split = self.config['time_split'] # init the number of time split
        
        self.odetime_splitted = ODETimeSplitter(self.time_split, self.config['K'])
        self.ode_block_1 = ODEBlock(ODEFunction(self.Graph), self.config['solver'], 0, self.odetime_splitted[0])
        self.ode_block_2 = ODEBlock(ODEFunction(self.Graph), self.config['solver'], self.odetime_splitted[0], self.odetime_splitted[1])
        self.ode_block_3 = ODEBlock(ODEFunction(self.Graph), self.config['solver'], self.odetime_splitted[1], self.odetime_splitted[2])
        self.ode_block_4 = ODEBlock(ODEFunction(self.Graph), self.config['solver'], self.odetime_splitted[2], self.config['K'])

    def get_time(self):
        ode_times=list(self.odetime_1)+ list(self.odetime_2)+ list(self.odetime_3)
        return ode_times
        
    def __dropout_x(self, x, keep_prob):
        size = x.size()
        index = x.indices().t()
        values = x.values()
        random_index = torch.rand(len(values)) + keep_prob
        random_index = random_index.int().bool()
        index = index[random_index]
        values = values[random_index]/keep_prob
        g = torch.sparse.FloatTensor(index.t(), values, size)
        return g
    
    def __dropout(self, keep_prob):
        if self.A_split:
            graph = []
            for g in self.Graph:
                graph.append(self.__dropout_x(g, keep_prob))
        else:
            graph = self.__dropout_x(self.Graph, keep_prob)
        return graph
    
    def computer(self):
        """
        propagate methods for LT-NCF
        """       
        users_emb = self.embedding_user.weight
        items_emb = self.embedding_item.weight
        all_emb = torch.cat([users_emb, items_emb])
        embs = [all_emb]
        
        """
        layers
        """
        all_emb_1 = self.ode_block_1(all_emb)
        all_emb_1 = all_emb_1 - all_emb
        embs.append(all_emb_1)

        all_emb_2 = self.ode_block_2(all_emb_1)
        all_emb_2 = all_emb_2 - all_emb_1
        embs.append(all_emb_2)

        all_emb_3 = self.ode_block_3(all_emb_2)
        all_emb_3 = all_emb_3 - all_emb_2
        embs.append(all_emb_3)
        
        all_emb_4 = self.ode_block_4(all_emb_3)
        all_emb_4 = all_emb_4 - all_emb_3
        embs.append(all_emb_4)

        embs = torch.stack(embs, dim=1)
        light_out = torch.mean(embs, dim=1)        

        users, items = torch.split(light_out, [self.num_users, self.num_items])
        return users, items
    
    def getUsersRating(self, users):
        all_users, all_items = self.computer()
        users_emb = all_users[users.long()]
        items_emb = all_items
        rating = self.f(torch.matmul(users_emb, items_emb.t()))
        return rating
    
    def getEmbedding(self, users, pos_items, neg_items):
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        pos_emb = all_items[pos_items]
        neg_emb = all_items[neg_items]
        users_emb_ego = self.embedding_user(users)
        pos_emb_ego = self.embedding_item(pos_items)
        neg_emb_ego = self.embedding_item(neg_items)
        return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego, neg_emb_ego
    
    def bpr_loss(self, users, pos, neg):
        (users_emb, pos_emb, neg_emb, 
        userEmb0,  posEmb0, negEmb0) = self.getEmbedding(users.long(), pos.long(), neg.long())
        reg_loss = (1/2)*(userEmb0.norm(2).pow(2) + 
                         posEmb0.norm(2).pow(2)  +
                         negEmb0.norm(2).pow(2))/float(len(users))
        pos_scores = torch.mul(users_emb, pos_emb)
        pos_scores = torch.sum(pos_scores, dim=1)
        neg_scores = torch.mul(users_emb, neg_emb)
        neg_scores = torch.sum(neg_scores, dim=1)
        
        loss = torch.mean(torch.nn.functional.softplus(neg_scores - pos_scores))
        
        return loss, reg_loss
       
    def forward(self, users, items):
        # compute embedding
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        inner_pro = torch.mul(users_emb, items_emb)
        gamma     = torch.sum(inner_pro, dim=1)
        return gamma

In [3]:
if world.dataset in ['gowalla', 'yelp2018', 'amazon-book']:
    dataset = dataloader.Loader(path="../data/"+world.dataset)

loading [../data/gowalla]
810128 interactions for training
217242 interactions for testing
gowalla Sparsity : 0.0008396216228570436
gowalla is ready to go


In [4]:
# Recmodel = register.MODELS[world.model_name](world.config, dataset)
Recmodel = LTOCF(world.config, dataset)

Recmodel = Recmodel.to(world.device)
if world.config['learnable_time'] == False:
    bpr = utils.BPRLoss(Recmodel, world.config)
elif world.config['learnable_time'] == True:
    bpr_t = utils.BPRLossT(Recmodel, world.config)

weight_file = utils.getFileName()
pretrained_weight_file = utils.getPretrainedFileName(world.config['pretrained_file_name'])
print(f"load and save to {pretrained_weight_file}")
if world.LOAD:
    try:
        Recmodel.load_state_dict(torch.load(pretrained_weight_file,map_location=torch.device('cpu')))
        world.cprint(f"loaded model weights from {pretrained_weight_file}")
    except FileNotFoundError:
        print(f"{pretrained_weight_file} not exists, start from beginning")
Neg_k = 1

save_name = time.strftime("%m-%d-%Hh%Mm-") + "-" + world.dataset + "-" + world.model_name + "-" + world.config['solver'] + "-adjoint_" + str(world.adjoint) + "-learnable_t_" + str(world.config['learnable_time']) + "-dual_res_" + str(world.config['dual_res']) + "-lr" + str(world.config['lr']) + "-lr_time" + str(world.config['lr_time']) + "-decay" + str(world.config['decay'])+ "-" + world.comment

# init tensorboard
if world.tensorboard:
    w : SummaryWriter = SummaryWriter(join(world.BOARD_PATH, save_name))
else:
    w = None
    world.cprint("not enable tensorflowboard")

use NORMAL distribution initilizer
loading adjacency matrix
successfully loaded...
don't split the matrix
lgn is already to go(dropout:0)
load and save to /home/bigdyl/jeongwhanchoi/tutorial21/LT-OCF/code/pretrain/ltocf


In [5]:
for epoch in range(world.TRAIN_epochs+1):
    start = time.time()
    if epoch %10 == 0:
        cprint("[TEST]")
        Procedure.Test(dataset, Recmodel, epoch, w, world.config['multicore'])

    if world.model_name == 'ltocf':
        if world.config['learnable_time'] == False:
            output_information = Procedure.BPR_train_original(dataset, Recmodel, bpr, epoch, neg_k=Neg_k,w=w)
            print(f'EPOCH[{epoch+1}/{world.TRAIN_epochs}] {output_information}')
        else:
            output_information, times_list= Procedure.BPR_train_ode(dataset, Recmodel, bpr_t, epoch, neg_k=Neg_k,w=w)
            print(f'EPOCH[{epoch+1}/{world.TRAIN_epochs}] {output_information}')
            print(times_list)

[TEST]
Inference time: 6.2452s
{'precision': array([0.00020932]), 'recall': array([0.00060648]), 'ndcg': array([0.00044858])}


KeyboardInterrupt: 

In [6]:
!python main.py --dataset="gowalla" --model="ltocf" --solver="rk4" --adjoint=False --learnable_time=False --dual_res=False --K=4 --lr=1e-3 --decay=1e-4 --topks="[20]" --comment="fixed_time" --tensorboard=1 --gpuid=0


██╗  ████████╗    ██████╗  ██████╗███████╗
██║  ╚══██╔══╝   ██╔═══██╗██╔════╝██╔════╝
██║     ██║█████╗██║   ██║██║     █████╗  
██║     ██║╚════╝██║   ██║██║     ██╔══╝  
███████╗██║      ╚██████╔╝╚██████╗██║     
╚══════╝╚═╝       ╚═════╝  ╚═════╝╚═╝     
                                                      

Current cuda device  0
>>SEED: 2020
loading [../data/gowalla]
810128 interactions for training
217242 interactions for testing
gowalla Sparsity : 0.0008396216228570436
gowalla is ready to go
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'K': 4.0,
 'bigdata': False,
 'bpr_batch_size': 2048,
 'decay': 0.0001,
 'dropout': 0,
 'dual_res': False,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'learnable_time': False,
 'lightGCN_n_layers': 3,
 'lr': 0.001,
 'lr_time': 0.0001,
 'multicore': 0,
 'pretrain': 0,
 'pretrained_file_name': 'ltocf',
 'solver': 'rk4',
 'test_u_batch_size': 100,
 'time_split': 4}
cores for test: 32
comment: fixed_time
tensorboard: 1
LOAD